In [1]:
import sys
import torch
import torchvision

from PIL import Image
from torchvision import transforms

# Focus: understand transfer learning with ResNet—how to load a pretrained model, preprocess input,
# move data/model to the correct device, and run inference.

# Load pretrained ResNet-50 and set to eval
model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
model.eval()

# Image I/O & preprocessing
filename = "cat.jpg"  # change to your image if needed
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # shape: [1, 3, 224, 224]

# Device selection & placement
# Detect available device ("cuda" > "mps" > "cpu") and set `device`
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch, "mps") and torch.mps.is_available():
    device = torch.device("mps")

# Move `input_batch` to `device`
input_batch = input_batch.to(device)

# Move `model` to `device`
model = model.to(device)

# Inference
with torch.no_grad():
    output = model(input_batch)  # logits shape: [1, 1000]

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Read ImageNet 1K class names
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top-5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/danilokacanski/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 57.5MB/s]


Egyptian cat 0.640246570110321
plastic bag 0.08858659118413925
tiger cat 0.08118936419487
sleeping bag 0.0715266764163971
tabby 0.029940564185380936
